Using SX generate parquet files for testing with join infrastructure. Also, using new SX frontend.

Constants:

In [1]:
# A Z to ee sample - Release 21
ds_name = r"mc16_13TeV:mc16_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.deriv.DAOD_PHYS.e3601_e5984_s3126_r10201_r10210_p5313"

What we'll need for imports

In [2]:
from servicex import ServiceXClient
from servicex import RucioDatasetIdentifier
from servicex import ResultFormat
from func_adl_servicex_xaodr22.event_collection import Event

# import logging
# logging.basicConfig(level=logging.DEBUG)

In [3]:
sx = ServiceXClient(backend="test4")
did = RucioDatasetIdentifier(ds_name, num_files=10)
ds_raw = sx.func_adl_dataset(
    did, codegen="atlasr21", title="Zee", result_format=ResultFormat.parquet, item_type=Event
)

from func_adl_servicex_xaodr22 import calib_tools
# ds = calib_tools.apply_calibration(ds_raw, "PHYS")
ds = calib_tools.query_update(ds_raw, calib_tools.default_config("PHYSLITE"))

In [4]:
electrons = (
    ds.Select(lambda e: e.Electrons('Electrons'))
    .Select(
        lambda e: e.Where(lambda e: (e.pt() / 1000 > 25.0) and (abs(e.eta()) < 2.5))
    )
    .Select(lambda e: e.Select(lambda e: e.pt() / 1000))
).as_files() #.generate_selection_string()

Output()

ServiceX Transform 0d6bb768-5727-443b-9e0b-d558e11454c8

Transforms completed successfully

In [5]:
import awkward as ak
pts = ak.from_parquet(electrons.file_list)
pts

<Array [{col1: []}, {...}, ..., {col1: [...]}] type='460000 * {col1: var * ...'>

# Query 1 - Electrons

Good healthy electrons that should be part of a Z mass peak: